In [0]:
with customer_orders as (
  SELECT customer_id, 
        date_format(date_trunc("month", min(first_purchase_date)),'yyyy-MM') as cohort_month,
        total_orders

  From workspace.bigquery_db_cohort_db.cohort_analysis
  group by customer_id, total_orders
  ),
  
  order_nums as (
  select *,
  CASE WHEN total_orders >= 2 THEN 1 END AS fct_2plus_orders,
  CASE WHEN total_orders >= 3 THEN 1 END AS fct_3plus_orders,
  CASE WHEN total_orders >= 4 THEN 1 END AS fct_4plus_orders  
  from customer_orders
  group by customer_id, cohort_month, total_orders
  )

  SELECT cohort_month, 
  ROUND(100 * sum(fct_2plus_orders)/ count(customer_id), 2) as num_2plus_orders,
  ROUND(100* sum(fct_3plus_orders)/ count(customer_id), 2) as num_3plus_orders,
  ROUND(100* sum(fct_4plus_orders)/ count(customer_id),2) as num_4plus_orders
  from order_nums
  group by cohort_month
  order by cohort_month